In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
api_key = " "

from apiclient.discovery import build
youtube = build('youtube', 'v3', developerKey=api_key)

import pandas as pd

ID = " "

box = [['Name', 'Comment', 'Time', 'Likes', 'Reply Count']]

def scrape_comments_with_replies():
  data = youtube.commentThreads().list(part='snippet', videoId-ID, maxResults='100', textFormat="plainText").execute()

  for i in data["items"]:

    name = i["snipper"]['topLevelComment']["snippet"]["authorDisplayName"]
    comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
    published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
    likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
    replies = i["snippet"]['totalReplyCount']

    box.append([name, comment, published_at, likes, replies])

    totalReplyCount = i["snippet"]['totalReplyCount']

    if totalReplyCount > 0:
      
      parent = i["snippet"]['topLevelComment']["id"]

      data2 = youtube.comments().list(part='snippet', maxResults='100', parentId=parent, textFormat="plainText").execute()

      for i in data2["items"]:
        name = i["snippet"]["authorDisplayName"]
        comment = i["snippet"]["textDisplay"]
        published_at = i["snippet"]['publishedAt']
        likes = i["snippet"]['likeCount']
        replies = " "

        box.append([name, comment, published_at, likes, replies])

  while ("nextPageToken" in data):

    data = youtube.commentThreads().list(part='snippet', videoId=ID, pageToken=data["nextPageToken"], maxResults='100', textFormat="plainText").execute()

    for i in data["items"]:
      name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
      comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
      published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
      likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
      replies = i["snippet"]['totalReplyCount']

      box.append([name, comment, published_at, likes, replies])

      totalReplyCount = i["snippet"]['totalReplyCount']

      if totalReplyCount > 0:

        parent = i["snippet"]['topLevelComment']["id"]

        data2 = youtube.comments().list(part='snippet', maxResults='100', parentId=parent, textFormat="plainText").execute()

        for i in data2["items"]:
          name = i["snippet"]["authorDisplayName"]
          comment = i["snippet"]["textDisplay"]
          published_at = i["snippet"]['publishedAt']
          likes = i["snippet"]['publishedAt']
          replies = ' '

          box.append([name, comment, published_at, likes, replies])

  df = pd.DataFrame({'Name': [i[0] for i in box], 'Comment': [i[1] for i in box], 'Time': [i[2] for i in box], 'Likes': [i[3] for i in box], 'Reply Count': [i[4] for i in box]})

  df.to_csv('youtube-comments.csv', index=False, header=False)

  return "Successful! Check the CSV File you have created"

In [ ]:
scrape_comments_with_replies()

In [ ]:
import pandas as pd
pd.options.displau.max_row = 10

df = pd.read_csv('youtube-comments.csv', index_col=0)
df

In [ ]:
df.columns

In [ ]:
df['Comment'].nunique()

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import nltk
nltk.download('stopwords')

In [ ]:
stop_words = stopwords.words('english')

stop_words[:10]

In [ ]:
def get_hashtags(text):
  hashtags = re.findall(r'\#\w+',text.lower())
  return hashtags
def get_mentions(text):
  mentions = re.findall(r'\@\w+',text.lower())
  return mentions

def remove_content(text):
  text = re.sub(r"http\S+","", text)
  text = re.sub(r'\S+\.com\S+','', text)
  text = re.sub(r'\@\w+', '', text)
  text = re.sub(r'\#\w+','', text)
  return text

def process_tweet(tweet):
  return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet.lower()).split())

def process_text(text, stem=False):
  text=remove_content(text)
  lemmatizer=WordNetLemmatizer()
  text = re.sub('[^A-Za-z', ' ', text.lower())
  text = re.sub(r'@[A-Za-Z0-9]+', ' ', str(text))
  text = re.sub(r'#', ' ', str(text))
  text = re.sub(r'RT[\s]+', '', str(text))
  text = re.sub(r'https?\/\/S+', ' ', str(text))
  text = re.sub(r'http\S+', '', str(text))
  text = re.sub(r'www\S+', '', str(text))
  text = re.sub(r'pic+', ' ', str(text))
  text = re.sub(r'com', '', str(text))
  text = re.sub(r"\bamp\b", ' ', text.lower())
  text = re.sub(r"\bco\b", ' ', text.lower())
  tokenized_text = word_tokenize(text)

  clean_text=[
      word for word in tokenized_text
      if (word not in stop_words and len(word)>1)
  ]
  if stem:
    clean_text=[stemmer.stem(word) for word in clean_text]
    clean_text = [lemmatizer.lemmatize(word) for word in clean_text]
    return ' '.join(clean_text)

def removeDupWithoutOrder(string):
  words = string.lower().split()
  return " ".join(sorted(set(words), key=words.index)).replace('OR', ' ').replace(' ',' ')

def remove_search(text, search_terms):
  query = text.lower()
  querywords = query.split()
  resultwords = [word for word in querywords if word.lower() not in search_terms]

from sklearn.feature_extraction.text import CountVectorizer
def plot_topn(sentences, ngram_range=(1,3), top=20, firstword=' '):
  c=CountVectorizer(ngram_range=ngram_range)
  X=c.fit_transform(sentences)
  words=pd.DataFrame(X.sum(axis=0),columns=c.get_featur_names()).T.sort_values(0,ascending=False).reset_index()
  res = words[words['index'].apply(lambda x: firstword in x)].head(top)
  pl=px.bar(res, x='index',y=0)
  pl.update_layout(yaxis_title='count',xaxis_title='Phrases')
  pl.show('png')

In [ ]:
import re
re.compile('<title>(.*)</title>')

In [ ]:
df['Comment']=df['Comment'].apply(lambda x: remove_content(x))

In [ ]:
df['cleaned_comments']=df['Comment'].apply(lambda x: process_tweet(x))

In [ ]:
df

In [ ]:
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

In [ ]:
comment_words = ' '
stopwords = set(STOPWORDS)

for val in df.cleaned_comments:
  val = str(val)

  tokens = val.split()

  for i in range(len(tokens)):
    tokens[i] = tokens[i].lower()
    comment_words += " ",join(tokens)+" "

wordcloud = WordCloud(width = 1000, height = 800, background_color='white', colormap='Set2', collocations=False, stopwords = stopwords, normalize_plurals=False, min_font_size=12).generate(comment_words)

plt.figure(figsize = (10,10), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
def getSubjectivity(text):
  return TextBlob(ste(text)).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(str(text)).sentiment.polarity

In [ ]:
dp.dropna(subset=['cleaned_comments'], inplace = True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['Subjectivity'] = df['cleaned_comments'].apply(getSubjectivity)
df['Polarity'] = df['cleaned_comments'].apply(getPolarity)
df.head()

In [ ]:
def get_Polarity_Analysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'

  def get_Subjectivity_Analysis(score):
    if score > 0:
      return 'Opinion'
    else:
      return 'Fact'

df['Analysis_Polarity'] = df['Polarity'].apply(get_Polarity_Analysis)

df['Analysis_Subjectivity'] = df['Subjectivity'].apply(get_Subjectivity_Analysis)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(font_scale=2)

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(df['Polarity'], df['Subjectivity'], c=df['Polarity'], s=100, cmap= 'RdY1Gn')

plt.xlim(-1.1, 1.1)
plt.ylim(-0.1, 1.1)
plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))

plt.title('Polarity Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis_Polarity'].value_counts().plot(kind = 'bar')
plt.show()

In [ ]:
plt.figure(figsize(7,5))

plt.title('Subjectivity Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis_Subjectivity'].value_counts().plot(kind = 'bar')
plt.show()